In [6]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("sudhirsingh27/electricity-consumption-based-on-weather-data")

print("Path to dataset files:", path)

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt

Path to dataset files: C:\Users\hanis\.cache\kagglehub\datasets\sudhirsingh27\electricity-consumption-based-on-weather-data\versions\1


In [7]:
DATA_PATH = "electricity_consumption_based_weather_dataset.csv"
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

display(df.head())

print("\nSummary statistics:")
display(df.describe(include="all"))

print("\nMissing values per column:")
display(df.isna().sum())

Shape: (1433, 6)

Columns:
['date', 'AWND', 'PRCP', 'TMAX', 'TMIN', 'daily_consumption']


,date,AWND,PRCP,TMAX,TMIN,daily_consumption
0,2006-12-16,2.5,0.0,10.6,5.0,1209.176
1,2006-12-17,2.6,0.0,13.3,5.6,3390.460
2,2006-12-18,2.4,0.0,15.0,6.7,2203.826
3,2006-12-19,2.4,0.0,7.2,2.2,1666.194
4,2006-12-20,2.4,0.0,7.2,1.1,2225.748



Summary statistics:


,date,AWND,PRCP,TMAX,TMIN,daily_consumption
count,1433,1418.000000,1433.000000,1433.000000,1433.000000,1433.000000
unique,1433,NaN,NaN,NaN,NaN,NaN
top,2006-12-16,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.642313,3.800488,17.187509,9.141242,1561.078061
std,NaN,1.140021,10.973436,10.136415,9.028417,606.819667
min,NaN,0.000000,0.000000,-8.900000,-14.400000,14.218000
25%,NaN,1.800000,0.000000,8.900000,2.200000,1165.700000
50%,NaN,2.400000,0.000000,17.800000,9.400000,1542.650000
75%,NaN,3.300000,1.300000,26.100000,17.200000,1893.608000



Missing values per column:


date                  0
AWND                 15
PRCP                  0
TMAX                  0
TMIN                  0
daily_consumption     0
dtype: int64

In [21]:
TARGET_COL = "daily_consumption"

def prepare_xy(df_in, target_col=TARGET_COL):
    """Drop missing rows, split into X and y."""
    df_clean = df_in.dropna()
    
    ##f target_col not in df_clean.columns:
       ## raise KeyError(f"Target column '{target_col}' not found. Available columns: {df_clean.columns.tolist()}")
    
    X = df_clean.drop(columns=["date","daily_consumption"])
    y = df_clean[target_col]
    
    # Keep only numeric columns
    ##X = X.select_dtypes(include=['number'])
    
    return X, y

def split_data(X, y, test_size=0.30, random_state= 42):
    """70/30 random train-test split."""
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

def compute_metrics(y_true, y_pred):
    """Return MSE, MAE, R^2."""
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "R^2": r2_score(y_true, y_pred),
    }

# Prepare data
X, y = prepare_xy(df, TARGET_COL)
X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.30, random_state= 42)

# Define models
models = {
    "Linear Regression": LinearRegression(),
    "Poly Degree 2": Pipeline([
        ("poly", PolynomialFeatures(degree=2, include_bias=False)),
        ("lr", LinearRegression())
    ]),
    "Poly Degree 3": Pipeline([
        ("poly", PolynomialFeatures(degree=3, include_bias=False)),
        ("lr", LinearRegression())
    ]),
    "Poly Degree 4": Pipeline([
        ("poly", PolynomialFeatures(degree=4, include_bias=False)),
        ("lr", LinearRegression())
    ]),
}

# Train and evaluate
results = []
for model_name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    
    # Predictions (MUST BE INSIDE THE LOOP)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Metrics (MUST BE INSIDE THE LOOP)
    train_metrics = compute_metrics(y_train, y_train_pred)
    test_metrics = compute_metrics(y_test, y_test_pred)
    
    # Append results (MUST BE INSIDE THE LOOP)
    results.append({
        "Model": model_name,
        "Train MSE": train_metrics["MSE"],
        "Train MAE": train_metrics["MAE"],
        "Train R²": train_metrics["R^2"],
        "Test MSE": test_metrics["MSE"],
        "Test MAE": test_metrics["MAE"],
        "Test R²": test_metrics["R^2"],
    })

# Display results
results_df = pd.DataFrame(results)
results_df

,Model,Train MSE,Train MAE,Train R²,Test MSE,Test MAE,Test R²
0,Linear Regression,272403.396174,384.465016,0.276000,2.481258e+05,375.404537,0.299333
1,Poly Degree 2,264765.769932,379.648753,0.296300,2.552685e+05,379.039083,0.279163
2,Poly Degree 3,259249.534870,375.952901,0.310961,2.656237e+05,385.235167,0.249922
3,Poly Degree 4,251909.339001,372.116566,0.330470,1.215149e+07,578.642201,-33.313844


Write a short, technical discussion that uses your results to answer:
• Which model generalizes best (best test performance), and what does that tell you about the
relationship between weather and electricity usage?
• Do polynomial models improve the fit compared to linear regression? If yes, why might electricity
consumption have nonlinear dependence on weather?
• If higher-degree models perform worse on the test set, explain this behavior using evidence from
metrics (e.g., train error decreases but test error increases).
• If none of the models achieve good test performance, provide at least two reasons supported by
your outputs (e.g., limited feature set, high noise, unmodeled drivers such as occupancy/behavior,
seasonal effects).


The best model is linear regression for this test. It has the largest R^2 value, being 0.299333. Even though this is the largest, it is not a great model. This tells us that the relationship between weather and electricity is linear. Polynomial models do not improve the fit for this data set. This is shown since for each degree as it increases, the R^2 value decreases, with the worst being poly degree 4 at -33.313844. there are signs of overfitting, especially in poly degree 4, which is shown by the largest test error of 1.24*10^7. this shows that the higher degree models fit noise in the training data. The limited feature set can cause the test performance to be worse. Weather is not the only thing that drives the amount of electricity usage. There is also 